In [0]:
# import libraries
from pyspark.sql.types import StringType, IntegerType, TimestampType, DateType, DoubleType, StructType, StructField
import requests
import json
import phonenumbers 
import re
import datetime
import pandas as pd
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import  col
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
from pyspark.sql.functions import monotonically_increasing_id

In [0]:
# schema for SCV User Table 
user_schema = StructType([
            StructField("id", IntegerType(), False),
            StructField("Userid", IntegerType(), True),
            StructField("SkinID", StringType(), True),
            StructField("username", StringType(), True),
            StructField("first_name", StringType(), True),
            StructField("last_name", StringType(), True),
            StructField("email", StringType(), True),
            StructField("gender", StringType(), True), 
            StructField("ip_address", StringType(), True), 
            StructField("RegDate", StringType(), True), 
            StructField("RegIP", StringType(), True), 
            StructField("LastIP", StringType(), True), 
            StructField("DOB", StringType(), True), 
            StructField("Postcode", StringType(), True), 
            StructField("MobilePhone", StringType(), True), 
            StructField("Landline", StringType(), True), 
            StructField("Address1", StringType(), True),
            StructField("City", StringType(), True),
            StructField("County", StringType(), True),
            StructField("Country", StringType(), True),
            StructField("SelfExcludedUntil", StringType(), True),
            StructField("Status", StringType(), True)])
            

In [0]:
# create the base directory to store csv files
dbutils.fs.rm("/FileStore/users",recurse=True)
dbutils.fs.mkdirs("/FileStore/users")
dbutils.fs.mkdirs("/FileStore/users/inprogress")
dbutils.fs.mkdirs("/FileStore/users/completed")


In [0]:
# list the contents of the base directory
dbutils.fs.ls("dbfs:/FileStore/users")


In [0]:
def fixUserRow(c):
    # get the Mobile field
    number = c.MobilePhone

    # initialize variables 
    is_valid_number = "N"
    clean_number = None
    number_type = None
    valid_mail = None

    p = None

    if number is not None:
        # Clean the Mobile Number first
        try:
            p = phonenumbers.parse(number, c.Country)

            if phonenumbers.is_valid_number(p):
                is_valid_number = "Y"
            elif phonenumbers.truncate_too_long_number(p):
                is_valid_number = "Y"
            else:
                is_valid_number = "N"

            clean_number = "%s%s" % (p.country_code, p.national_number)
            
        except:
            p = None

    # clean up PhoneNumber
    phone_no = c.Landline
    if phone_no is not None:
      phone_no = phone_no.replace('-', '')
      if (len(phone_no) != 10):
        phone_no = None
    
    # validate Email 
    if re.match(r"^[A-Za-z0-9\.\+_-]+@[A-Za-z0-9\._-]+\.[a-zA-Z]*$", c.email):
      valid_mail = c.email
    
    return Row( 
		id = c.id, 
        Userid = c.Userid, 			
        SkinID = c.SkinID,
        username = c.username,
        first_name = c.first_name, 		
        last_name = c.last_name,	
        email = valid_mail,			
        gender = c.gender,			
        ip_address = c.ip_address,
        RegDate = c.RegDate,
        RegIP = c.RegIP,
		LastIP = c.LastIP,			
		DOB = c.DOB,			
		Postcode = c.Postcode,		
		MobilePhone = clean_number, 	
		Landline = phone_no, 		
		Address1 = c.Address1,		
        City = c.City, 			
		County = c.County,			
		Country = c.Country, 		
        SelfExcludedUntil = c.SelfExcludedUntil,
		Status = c.Status			
    )


In [0]:
# insert matching records into UserSCV table
def insertNewVersionOfUser(tableName):
  df = spark.sql("select  * from " + tableName)
  dateTimeStr = datetime.datetime.today().strftime("%m-%d-%Y %H:%M:%S")

  # select max of id from userSCV table
  lv = sqlContext.sql("select max(ID) as lastVal from UserSCV").collect()
  lastValue = lv[0]["lastVal"]
  df_userSCV = df.select("ID", \
                         "Userid1", \
                         "SkinID1", \
                         "username1", \
                         "first_name1", \
                         "last_name1", \
                         "email1", \
                         "gender1", "ip_address1", "RegDate1", "RegIP1", \
                         "LastIP1", "DOB1", "Postcode1", "MobilePhone1", "Landline1", \
                         "Address11", "City1", "County1", "Country1", \
                         "SelfExcludedUntil1", "Status1", \
                         "EntityId", \
                         "OriginalEmail", \
                         "OriginalFirstname", \
                         "OriginalLastname", \
                         "OriginalRegDate", \
                         "OriginalDOB", \
                         "OriginalPostcode", \
                         "OriginalMobilePhone", \
                         "OriginalAddress1", \
                         "OriginalCity", \
                         "Firstname_Lastname_RegIP", \
                         "Firstname_Lastname_LastIP", \
                         "Firstname_Lastname_Username", \
                         "Firstname_DOB_City",\
                         "Firstname_Postcode", \
                         "Firstname_Mobilephone", \
                         "DOB_Postcode",  \
                         "Address1_Postcode", \
                         "Firstname_Lastname_Address1_City")
  #df_userSCV = df_userSCV.withColumnRenamed("ID", "RelatedID") 
  df_userSCV = df_userSCV.withColumn("RelatedID", col("ID"))
  df_userSCV = df_userSCV.withColumn("Load_date", lit(dateTimeStr))
  df_userSCV = df_userSCV.withColumn("LastModifiedDate", lit(dateTimeStr))
  df_userSCV = df_userSCV.withColumn("CompareStatus", lit(0))
   
  #df_userSCV = df_userSCV.withColumn("ID", monotonically_increasing_id() + lastValue)
  df_userSCV = df_userSCV.select("ID", \
                        col("Userid1").alias("Userid"), col("SkinID1").alias("SkinID"), \
                        col("username1").alias("username"), col("first_name1").alias("first_name"), \
                        col("last_name1").alias("last_name"), col("email1").alias("email"), \
                        col("gender1").alias("gender"), col("ip_address1").alias("ip_address"), \
                        col("RegDate1").alias("RegDate"), col("RegIP1").alias("RegIP"), \
                        col("LastIP1").alias("LastIP"), col("DOB1").alias("DOB"), \
                        col("Postcode1").alias("Postcode"), col("MobilePhone1").alias("MobilePhone"), \
                        col("Landline1").alias("Landline"), col("Address11").alias("Address1"), \
                        col("City1").alias("City"), col("County1").alias("County"), \
                        col("Country1").alias("Country"), col("SelfExcludedUntil1").alias("SelfExcludedUntil"), \
                        col("Status1").alias("Status"), \
                         "RelatedID", \
                         "EntityId", \
                         "OriginalEmail", \
                         "OriginalFirstname", \
                         "OriginalLastname", \
                         "OriginalRegDate", \
                         "OriginalDOB", \
                         "OriginalPostcode", \
                         "OriginalMobilePhone", \
                         "OriginalAddress1", \
                         "OriginalCity", \
                         "Firstname_Lastname_RegIP", \
                         "Firstname_Lastname_LastIP", \
                         "Firstname_Lastname_Username", \
                         "Firstname_DOB_City",\
                         "Firstname_Postcode", \
                         "Firstname_Mobilephone", \
                         "DOB_Postcode",  \
                         "Address1_Postcode", \
                         "Firstname_Lastname_Address1_City", \
                         "Load_date", \
                         "LastModifiedDate",\
                         "CompareStatus")

  df_userSCV.write.insertInto("UserSCV")
  
  

In [0]:
def getDataFrameFromCSV(csvFileLocation, schema):
   
  df_new_load = sqlContext.read.format('com.databricks.spark.csv') \
                          .options(header=True).schema( schema).load(csvFileLocation)
  from pyspark.sql.functions import col

  
  # cleanse the data
  df_user_updated1 = df_new_load.rdd.map(lambda c: fixUserRow(c))
  # change the column type now
  df_new = sqlContext.createDataFrame(df_user_updated1, user_schema)
  df_new = df_new.select (col("ID").alias("ID1"), col("Userid").alias("Userid1"), col("SkinID").alias("SkinID1"), \
                        col("username").alias("username1"), col("first_name").alias("first_name1"), \
                        col("last_name").alias("last_name1"), col("email").alias("email1"), \
                        col("gender").alias("gender1"), col("ip_address").alias("ip_address1"), \
                        col("RegDate").alias("RegDate1"), col("RegIP").alias("RegIP1"), \
                        col("LastIP").alias("LastIP1"), col("DOB").alias("DOB1"), \
                        col("Postcode").alias("Postcode1"), col("MobilePhone").alias("MobilePhone1"), \
                        col("Landline").alias("Landline1"), col("Address1").alias("Address11"), \
                        col("City").alias("City1"), col("County").alias("County1"), \
                        col("Country").alias("Country1"), col("SelfExcludedUntil").alias("SelfExcludedUntil1"), \
                        col("Status").alias("Status1")) 
  return df_new
  

In [0]:
def createOutputTable(tableName):
  # create output table
  df = spark.sql("select * from " + tableName)
  dateTimeStr = datetime.datetime.today().strftime("%m-%d-%Y %H:%M:%S")


  userSCV =  df.withColumn("ID", F.monotonically_increasing_id()) \
    .withColumn("RelatedID", lit(-1).cast(IntegerType())) 
  userSCV = userSCV.withColumn("EntityId", col("ID")) 

  # rename columns 
  userSCV = userSCV.withColumn("OriginalEmail", col("email")) 
  userSCV = userSCV.withColumn("OriginalFirstname", col("first_name")) 
  userSCV = userSCV.withColumn("OriginalLastname", col("last_name")) 
  userSCV = userSCV.withColumn("OriginalRegDate", col("RegDate"))
  userSCV = userSCV.withColumn("OriginalDOB", col("DOB"))
  userSCV = userSCV.withColumn("OriginalPostcode", col("Postcode"))             
  userSCV = userSCV.withColumn("OriginalMobilePhone", col("MobilePhone"))
  userSCV = userSCV.withColumn("OriginalAddress1", col("Address1"))            
  #userSCV = userSCV.withColumn("OriginalAddress2", col("Address2"))            
  userSCV = userSCV.withColumn("OriginalCity", col("City"))
  userSCV = userSCV.withColumn("Firstname_Lastname_RegIP", F.concat(col('first_name'),lit('_'), col('last_name'), lit('_'),col('RegIP') ))       
  userSCV = userSCV.withColumn("Firstname_Lastname_LastIP", \
                               F.concat(col('first_name'),lit('_'), col('last_name'), lit('_'),col('LastIP') ))
  userSCV = userSCV.withColumn("Firstname_Lastname_Username", \
                               F.concat(col('first_name'),lit('_'), col('last_name'), lit('_'),col('Username') ))
  userSCV = userSCV.withColumn("Firstname_DOB_City", F.concat(col('first_name'),lit('_'), col('DOB'), lit('_'),col('City') ))
  userSCV = userSCV.withColumn("Firstname_Postcode", F.concat(col('first_name'),lit('_'), col('Postcode')  )) 
  userSCV = userSCV.withColumn("Firstname_Mobilephone", F.concat(col('first_name'),lit('_'), col('MobilePhone')  ))          
  userSCV = userSCV.withColumn("DOB_Postcode", F.concat(col('DOB'),lit('_'), col('Postcode')  )) 
  userSCV = userSCV.withColumn("Address1_Postcode", F.concat(col('Address1'),lit('_'), col('Postcode')  ))              
  userSCV = userSCV.withColumn("Firstname_Lastname_Address1_City", \
                               F.concat(col('first_name'),lit('_'), col('last_name'), lit('_'),col('Address1'), lit('_'), col('City') ))
  userSCV = userSCV.withColumn("Load_date", lit(dateTimeStr))
  userSCV = userSCV.withColumn("LastModifiedDate", lit(dateTimeStr))
  userSCV = userSCV.withColumn("CompareStatus", lit(0))
  userSCV = userSCV.withColumn("CompareStatus", lit(None).cast(StringType()))
  # Create a HIVE table to save Data fro Dataframe 
  if (len(spark.sql("SHOW TABLES LIKE '" + "UserSCV"+ "'").collect()) == 1):
    userSCV.write.insertInto("UserSCV")
  else:
    userSCV.write.saveAsTable("UserSCV")



In [0]:
def compareData(tableName):
  spark.sql("REFRESH TABLE  " + tableName)
  df_temp = spark.sql ("select * from " + tableName)
  count = df_temp.count()
  if (count > 0):
    insertNewVersionOfUser(tableName)
  

In [0]:
def processUserInfo(filePath):
  # Read the csv file as example
  print("-----------------------------------------------------------------------------------")
  print("Reading csv file from :" + filePath)
  df = sqlContext.read.format('com.databricks.spark.csv') \
              .options(header=True).schema( user_schema).load(filePath)
  # cleanse the data
  # Standardise the telephone number to take away -, ) from input data 

  df_user_updated = df.rdd.map(lambda c: fixUserRow(c))
  # change the column type now
  df_user = sqlContext.createDataFrame(df_user_updated, user_schema)
  # Insert into intermediate table
  # check if table exists
  if (len(spark.sql("SHOW TABLES LIKE '" + "users_load"+ "'").collect()) == 1):
    df_user.write.insertInto("users_load")
  else:
    df_user.write.saveAsTable("users_load")

    
  
  
  print("After saving data to userload")
  # check if UserSCV table exists:
  if (len(spark.sql("SHOW TABLES LIKE '" + "UserSCV"+ "'").collect()) == 1) :
     # compare the data with existing data in UserSCV
    userSCV = spark.sql("select * from UserSCV")
    # 1. Rename the base columns 
    df_new =  getDataFrameFromCSV(filePath, user_schema)
    # 2. compare the data
    # check for the minimal condition
    # whether firstName + IP equals
    print("1. checking for firstName + IP")
    df_criteria_min = userSCV.join(df_new, (userSCV.first_name == df_new.first_name1) & (userSCV.ip_address == df_new.ip_address1) )
    df_criteria_min.createOrReplaceTempView("c1_FN_IP") 
    compareData("c1_FN_IP")
    
    # This is to check  criteria: FirstName + username 
    print("1. checking for firstName + username") 
    #df_new =  getDataFrameFromCSV(csvFilePath_new, user_schema)
    df_criteria_fn_username = userSCV.join(df_new, (userSCV.first_name == df_new.first_name1) & (userSCV.username == df_new.username1) )
    df_criteria_fn_username.createOrReplaceTempView("c1_FN_username") 
    compareData("c1_FN_username")
    df_user.createOrReplaceTempView("output_user")
    createOutputTable("output_user")
    
  # create the UserSCV table
  else:
    print("UserSCV table do not exist; hence creating it")
    df_user.createOrReplaceTempView("output_user")
    createOutputTable("output_user")
    
  
  print("-----------------------------------------------------------------------------------")

 

In [0]:
def fixUserRow(c):
    # get the Mobile field
    number = c.MobilePhone

    # initialize variables 
    is_valid_number = "N"
    clean_number = None
    number_type = None
    valid_mail = None

    p = None

    if number is not None:
        # Clean the Mobile Number first
        try:
            p = phonenumbers.parse(number, c.Country)

            if phonenumbers.is_valid_number(p):
                is_valid_number = "Y"
            elif phonenumbers.truncate_too_long_number(p):
                is_valid_number = "Y"
            else:
                is_valid_number = "N"

            clean_number = "%s%s" % (p.country_code, p.national_number)
            
        except:
            p = None

    # clean up PhoneNumber
    phone_no = c.Landline
    if phone_no is not None:
      phone_no = phone_no.replace('-', '')
      if (len(phone_no) != 10):
        phone_no = None
    
    # validate Email 
    if re.match(r"^[A-Za-z0-9\.\+_-]+@[A-Za-z0-9\._-]+\.[a-zA-Z]*$", c.email):
      valid_mail = c.email
    
    return Row( 
		id = c.id, 
        Userid = c.Userid, 			
        SkinID = c.SkinID,
        username = c.username,
        first_name = c.first_name, 		
        last_name = c.last_name,	
        email = valid_mail,			
        gender = c.gender,			
        ip_address = c.ip_address,
        RegDate = c.RegDate,
        RegIP = c.RegIP,
		LastIP = c.LastIP,			
		DOB = c.DOB,			
		Postcode = c.Postcode,		
		MobilePhone = clean_number, 	
		Landline = phone_no, 		
		Address1 = c.Address1,		
        City = c.City, 			
		County = c.County,			
		Country = c.Country, 		
        SelfExcludedUntil = c.SelfExcludedUntil,
		Status = c.Status			
    )


In [0]:
import schedule
import time
import requests
import datetime

# create the base directory to store csv files
dbutils.fs.rm("/FileStore/users",recurse=True)
dbutils.fs.mkdirs("/FileStore/users")
dbutils.fs.mkdirs("/FileStore/users/inprogress")
dbutils.fs.mkdirs("/FileStore/users/completed")

 
def job():
  print("calling CSV load function")
  url = "https://my.api.mockaroo.com/users_load.json?key=6af9c3e0"
  import pandas as pd
  df = spark.createDataFrame(pd.read_csv(url))
  ts = time.time()
  st = datetime.datetime.fromtimestamp(ts).strftime('%Y_%m_%d_%H_%M')
  fileName = '/FileStore/users/inprogress/'+ st + '.tmp'
  fileprefix = '/FileStore/users/inprogress/'
  df.coalesce(1).write.format("com.databricks.spark.csv") \
    .option("header", True) \
    .option("quote", "") \
    .save(fileName)  #saved to the FileStore
    
  fileList =  dbutils.fs.ls(fileName)

  csvFileLocation = ''
  for fileInfo in fileList:   
    if ".csv" in fileInfo.path:
      print("this file is csv file.." )
      print(fileInfo.path)
      csvFileLocation = fileprefix + fileInfo.name
      
      dbutils.fs.cp(fileInfo.path,fileprefix)
      dbutils.fs.rm(fileName,recurse=True)
  
  #if (len(csvFileLocation) >0):
  #  processUserInfo(csvFileLocation)
  #  dbutills.fs.mv(csvFileLocation, '/FileStore/users/completed/')
      
schedule.every(10).seconds.do(job)
 

while True:
    schedule.run_pending()
    time.sleep(1)

calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_18.tmp/part-00000-tid-6316412808255974382-58b10a30-bfc5-4068-be03-255c53270f22-0-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_18.tmp/part-00000-tid-185255574446068700-a25437e2-ceb5-4470-a300-bdfb64f7f949-1-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_19.tmp/part-00000-tid-3152717898253485723-5e785086-5e03-446b-a755-b95610197d41-2-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_19.tmp/part-00000-tid-3312424275174312263-2a008a45-d514-4880-a509-9bc38bfd21dc-6-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_19.tmp/part-00000-tid-8301882247220086355-e75912e4-738b-417f-b251-5957377819fe-10-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_19.tmp/part-00000-tid-1821240793519352593-9b5da1a4-e6ee-4ad1-850f-647261ca440d-14-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_20.tmp/part-00000-tid-4986515206526566214-93eceeb2-2b8b-4fec-9858-aec311ce876f-18-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_20.tmp/part-00000-tid-3147340080791193434-9bb60a0c-3b09-4cb7-975b-bb804dc40d05-22-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_20.tmp/part-00000-tid-7680131892817473880-7189c01d-304d-4e45-ad66-84b27cf6a8f3-26-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_20.tmp/part-00000-tid-8763862965712818502-017b0772-764b-46a9-9be7-4d9f8c9d5c86-30-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_21.tmp/part-00000-tid-2410967365504892243-54efaff1-eb8d-4065-bae4-a63460216c60-34-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_21.tmp/part-00000-tid-1782553679707122239-fe8c820f-e70d-4e0b-ad82-4ec9d54d6955-38-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_21.tmp/part-00000-tid-3345097634220269823-6f9ea4dd-fc22-42f8-8f1c-2089f1e47eb7-42-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_21.tmp/part-00000-tid-1813933487326690910-ec62c50c-6e3b-4ac4-9ae7-82a34dd080f2-46-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_21.tmp/part-00000-tid-2619521073048134800-5f7dd494-02fe-4534-a490-a93a7a34a0ad-50-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_22.tmp/part-00000-tid-1969133709063988885-9ecef208-38bd-421d-a491-79fdb7740bb9-54-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_22.tmp/part-00000-tid-1887597871054634786-88f5d245-d846-4604-bf71-f4b74b61dbd6-58-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_22.tmp/part-00000-tid-3549754338760075600-2263b59e-43f5-486b-84e8-a93ed8f7634d-62-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_22.tmp/part-00000-tid-8684791076371654167-75980760-0f24-4aa2-be06-6979cb3b372b-66-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_23.tmp/part-00000-tid-3383420605636232292-8773d3a2-4f09-4e1d-ac1c-3a5e362432b4-70-c000.csv
calling CSV load function
this file is csv file..
dbfs:/FileStore/users/inprogress/2018_06_20_03_23.tmp/part-00000-tid-3103639318417769381-22abf6bb-9bbf-4d13-9aa1-9bfc65840516-74-c000.csv
calling CSV load function
this file is csv file..
dbfs:/F

--------------------------------------------------------------------------- 
 HTTPError Traceback (most recent call last)
 <command-689586480842088> in <module> () 
 42 
 43 while True : 
 ---> 44 schedule . run_pending ( ) 
 45 time . sleep ( 1 ) 

 /databricks/python/local/lib/python2.7/site-packages/schedule/__init__.pyc in run_pending () 
 491 : data : ` default scheduler instance < default_scheduler > ` . 
 492 """
 --> 493 default_scheduler . run_pending ( ) 
 494 
 495 

 /databricks/python/local/lib/python2.7/site-packages/schedule/__init__.pyc in run_pending (self) 
 76 runnable_jobs = ( job for job in self . jobs if job . should_run ) 
 77 for job in sorted ( runnable_jobs ) : 
 ---> 78 self . _run_job ( job ) 
 79 
 80 def run_all ( self , delay_seconds = 0 ) : 

 /databricks/python/local/lib/python2.7/site-packages/schedule/__init__.pyc in _run_job (self, job) 
 129 
 130 def _run_job ( self , job ) : 
 --> 131 ret = job . run ( ) 
 132 if isinstance ( ret , CancelJob ) or ret is CancelJob : 
 133 self . cancel_job ( job ) 

 /databricks/python/local/lib/python2.7/site-packages/schedule/__init__.pyc in run (self) 
 409 """
 410 logger . info ( 'Running job %s' , self ) 
 --> 411 ret = self . job_func ( ) 
 412 self . last_run = datetime . datetime . now ( ) 
 413 self . _schedule_next_run ( ) 

 <command-689586480842088> in job () 
 15 url = "https://my.api.mockaroo.com/users_load.json?key=6af9c3e0" 
 16 import pandas as pd 
 ---> 17 df = spark . createDataFrame ( pd . read_csv ( url ) ) 
 18 ts = time . time ( ) 
 19 st = datetime . datetime . fromtimestamp ( ts ) . strftime ( '%Y_%m_%d_%H_%M' ) 

 /databricks/python/local/lib/python2.7/site-packages/pandas/io/parsers.pyc in parser_f (filepath_or_buffer, sep, delimiter, header, names, index_col, usecols, squeeze, prefix, mangle_dupe_cols, dtype, engine, converters, true_values, false_values, skipinitialspace, skiprows, nrows, na_values, keep_default_na, na_filter, verbose, skip_blank_lines, parse_dates, infer_datetime_format, keep_date_col, date_parser, dayfirst, iterator, chunksize, compression, thousands, decimal, lineterminator, quotechar, quoting, escapechar, comment, encoding, dialect, tupleize_cols, error_bad_lines, warn_bad_lines, skipfooter, skip_footer, doublequote, delim_whitespace, as_recarray, compact_ints, use_unsigned, low_memory, buffer_lines, memory_map, float_precision) 
 644 skip_blank_lines=skip_blank_lines)
 645 
 --> 646 return _read ( filepath_or_buffer , kwds ) 
 647 
 648 parser_f . __name__ = name 

 /databricks/python/local/lib/python2.7/site-packages/pandas/io/parsers.pyc in _read (filepath_or_buffer, kwds) 
 373 filepath_or_buffer, _, compression = get_filepath_or_buffer(
 374 filepath_or_buffer , encoding , 
 --> 375 compression=kwds.get('compression', None))
 376 kwds['compression'] = (inferred_compression if compression == 'infer'
 377 else compression)

 /databricks/python/local/lib/python2.7/site-packages/pandas/io/common.pyc in get_filepath_or_buffer (filepath_or_buffer, encoding, compression) 
 236 
 237 if _is_url ( filepath_or_buffer ) : 
 --> 238 req = _urlopen ( str ( filepath_or_buffer ) ) 
 239 if compression == 'infer' : 
 240 content_encoding = req . headers . get ( 'Content-Encoding' , None ) 

 /usr/lib/python2.7/urllib2.pyc in urlopen (url, data, timeout, cafile, capath, cadefault, context) 
 152 else : 
 153 opener = _opener 
 --> 154 return opener . open ( url , data , timeout ) 
 155 
 156 def install_opener ( opener ) : 

 /usr/lib/python2.7/urllib2.pyc in open (self, fullurl, data, timeout) 
 433 for processor in self . process_response . get ( protocol , [ ] ) : 
 434 meth = getattr ( processor , meth_name ) 
 --> 435 response = meth ( req , response ) 
 436 
 437 return response 

 /usr/lib/python2.7/urllib2.pyc in http_response (self, request, response) 
 546 if not ( 200 <= code < 300 ) : 
 547 response = self.parent.error(
 --> 548 'http', request, response, code, msg, hdrs)
 549 
 550 return response 

 /usr

In [0]:
spark.sql("REFRESH TABLE UserSCV")
df1 = spark.sql ("select  * from UserSCV")

df1.count()